In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import datetime
import shutil
import difflib
from pulp import *

today = date.today()
today = today - datetime.timedelta(days=1)
today = today.strftime("%m.%d.20%y")
today

'12.31.2021'

In [7]:
downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis'
daily_file = "final_PlayerPool_" + str(today) + ".csv"

os.chdir(data_loc)

df = pd.read_csv(daily_file)

In [1]:
df.head()
df['Roster Position'] = df['Roster Position'].str.split('/')
explode = df.explode('Roster Position')
explode.head()

NameError: name 'df' is not defined

In [9]:
# Get a list of players
players = list(df['Name'])

# Initialize Dictionaries for Salaries and Positions
salaries = dict(zip(players, df['Salary']))
positions = dict(zip(players, df['Roster Position']))

# Dictionary for Projected Score for each player
project_points = dict(zip(players, df['Expected_Points']))

# Set Players to Take either 1 or 0 values (owned or not)
player_vars = LpVariable.dicts("Player", players, lowBound=0, upBound=1, cat='Integer')

In [10]:
total_score = LpProblem("Fantasy_Points_Problem", LpMaximize)

total_score += lpSum([project_points[i] * player_vars[i] for i in player_vars])

total_score += lpSum([salaries[i] * player_vars[i] for i in player_vars]) <= 50000

In [12]:
# Get indices of players for each position
pg = [p for p in positions.keys() if positions[p] == 'PG']
sg = [p for p in positions.keys() if positions[p] == 'SG']
g =  [p for p in positions.keys() if positions[p] == 'G']
sf = [p for p in positions.keys() if positions[p] == 'SF']
pf = [p for p in positions.keys() if positions[p] == 'PF']
f = [p for p in positions.keys() if positions[p] == 'F']
c =  [p for p in positions.keys() if positions[p] == 'C']
# Set Constraints
total_score += lpSum([player_vars[i] for i in pg]) == 1
total_score += lpSum([player_vars[i] for i in sg]) == 1
total_score += lpSum([player_vars[i] for i in g]) == 1
total_score += lpSum([player_vars[i] for i in sf]) == 1
total_score += lpSum([player_vars[i] for i in f]) == 1
total_score += lpSum([player_vars[i] for i in pf]) == 1
total_score += lpSum([player_vars[i] for i in c]) == 1

In [19]:
for v in total_score.variables():
    if v.varValue > 0:
        print(v.name)
        
    else:
        break

TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [21]:
df.dtypes

Rank                 int64
Name                object
Team                object
Pos                 object
Opp                 object
p                  float64
3                  float64
r                  float64
a                  float64
s                  float64
b                  float64
fg%                float64
fga                float64
ft%                float64
fta                float64
to                 float64
fgm                float64
ftm                float64
Expected_Points    float64
Salary               int64
Roster Position    float64
Value              float64
dtype: object